# Building Complex SQL Queries

Continuing to use the Chinook database we will further the exploration of SQL into building complex queries.

<img src = "chinook-schema.svg" width=500>

## First Problem

Create a query that shows summary data for every playlist in the Chinook database:

Use a WITH clause to create a named subquery with the following info:
1. The unique ID for the playlist.
2. The name of the playlist.
3. The name of each track from the playlist.
4. The length of each track in seconds.

Your final table should have the following columns, in order:
1. playlist_id - the unique ID for the playlist.
2. playlist_name - The name of the playlist.
3. number_of_tracks - A count of the number of tracks in the playlist.
4. length_seconds - The sum of the length of the playlist in seconds.

The results should be sorted by playlist_id in ascending order.


In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
WITH subquery AS 
(
    SELECT pl.playlist_id pl_id,
           pl.name playlist_name,
           tr.name track_name,
           CAST(tr.milliseconds AS FLOAT) / 1000 as length
    FROM playlist pl
    LEFT JOIN playlist_track pt ON pl.playlist_id = pt.playlist_id
    LEFT JOIN track tr ON tr.track_id = pt.track_id
)

SELECT pl_id playlist_id,
       playlist_name,
       COUNT(track_name) number_of_tracks,
       SUM(length) length_seconds
FROM subquery
GROUP BY 1,2
ORDER BY 1;

 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,877683.0829999988
2,Movies,0,None
3,TV Shows,213,501094.95700000005
4,Audiobooks,0,None
5,90’s Music,1477,398705.153
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,877683.0829999988
9,Music Videos,1,294.294
10,TV Shows,213,501094.95700000005


## Second Query
    
1. Create a view called customer_gt_90_dollars:
    1. The view should contain the columns from customers, in their original order.
    2. The view should contain only customers who have purchased more than $90 in tracks from the store.
2. After the SQL query that creates the view, write a second query to display your newly created view: SELECT * FROM chinook.customer_gt_90_dollars;.
    1. Make sure you use a semicolon (;) to indicate the end of each query.


First I want to drop the two views that are built, so that running the following code won't throw an error.

In [3]:
%%sql 
DROP VIEW customer_gt_90_dollars;
DROP VIEW customer_usa;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such view: customer_gt_90_dollars
[SQL: DROP VIEW customer_gt_90_dollars;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [4]:
%%sql
CREATE VIEW customer_gt_90_dollars AS
    SELECT c.*
    FROM customer c
    INNER JOIN invoice i
    ON i.customer_id = c.customer_id
    GROUP BY c.customer_id
    HAVING SUM(i.total) > 90;
    
SELECT *
FROM customer_gt_90_dollars;

 * sqlite:///chinook.db
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


In [5]:
%%sql
CREATE VIEW customer_usa AS 
    SELECT *
    FROM customer
    WHERE country = 'USA';
    
SELECT *
FROM customer_usa;

 * sqlite:///chinook.db
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


In order to get a bigger picture, I want to query the sqlite master so that I can see what is in the database.

In [6]:
%%sql
SELECT name, type
FROM sqlite_master;

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [7]:
%%sql
SELECT *
FROM customer_usa
UNION
SELECT *
FROM customer_gt_90_dollars;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


## Practicing Row Combinations Between Tables

Because customers_gt_90_dollars and customers_usa have the same columns, I can combine them into one query that returns customers who have either spent over 90 dollars or are from the USA.

Similarly to **joins** for working with data tables with the same rows, we can combine multiple tables with the same columns using logical *ands*, *ors*, and *nots*. 

1. $A \cup B$ is $A$ UNION $B$ 
2. $A \cap B$ is $A$ INTERSECTION $B$
3. $A-B$ is $A$ EXCEPT $B$

In [8]:
%%sql
SELECT *
FROM customer_gt_90_dollars

UNION

SELECT *
FROM customer_usa;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


<img src = "chinook-schema.svg" width=500>

In [9]:
%%sql
WITH us_gt_90 AS (
    SELECT *
    FROM customer_usa
    INTERSECT
    SELECT *
    FROM customer_gt_90_dollars)

SELECT e.first_name || ' ' || e.last_name employee_name,
       COUNT(u.support_rep_id) customers_usa_gt_90
FROM employee e
LEFT JOIN us_gt_90 u
ON u.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1
ORDER BY 1;
       

 * sqlite:///chinook.db
Done.


employee_name,customers_usa_gt_90
Jane Peacock,0
Margaret Park,2
Steve Johnson,2


In [10]:
%%sql
SELECT *
FROM customer_usa
INTERSECT
SELECT *
FROM customer_gt_90_dollars;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


In [11]:
%%sql
SELECT * FROM employee

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [12]:
%%sql
WITH customer_india AS
    (SELECT * 
     FROM customer
     WHERE country = 'India'),
     customer_total AS 
    (SELECT c.*,
            SUM(i.total) total_purchases
     FROM customer c
     LEFT JOIN invoice i ON c.customer_id = i.customer_id
     GROUP BY c.customer_id)
    
SELECT i.first_name || ' ' || i.last_name customer_name,
       t.total_purchases total_purchases
FROM customer_india i
LEFT JOIN customer_total t
ON i.customer_id = t.customer_id
ORDER BY 1;

 * sqlite:///chinook.db
Done.


customer_name,total_purchases
Manoj Pareek,111.86999999999999
Puja Srivastava,71.28


Create a query to find the customer from each country that has spent the most money at our store, ordered alphabetically by country. Your query should return the following columns, in order:

1. country - The name of each country that we have a customer from.
2. customer_name - The first_name and last_name of the customer from that country with the most total purchases, separated by a space, eg Luke Skywalker.
3. total_purchased - The total dollar amount that customer has purchased.


Needed Info:
1. Total Money Spent by each Customer including their country

In [13]:
%%sql
DROP VIEW customer_total;

CREATE VIEW customer_total AS
SELECT c.first_name || ' ' || c.last_name customer_name,
       c.country,
       SUM(i.total) total_purchases
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1;

SELECT * FROM customer_total
LIMIT 10;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such view: customer_total
[SQL: DROP VIEW customer_total;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [14]:
%%sql
SELECT customer_name,
       country, 
       ROUND(MAX(total_purchases),2) total_purchased
FROM customer_total
GROUP BY country;

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such table: customer_total
[SQL: SELECT customer_name,
       country, 
       ROUND(MAX(total_purchases),2) total_purchased
FROM customer_total
GROUP BY country;]
(Background on this error at: http://sqlalche.me/e/e3q8)
